In [4]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# Pandas NumPy and TensorFlow
import pandas as pd
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from utils import utils_proj, vocabulary

from sklearn.model_selection import train_test_split

import lstm; reload(lstm)
import matplotlib.pyplot as plt
% matplotlib inline

In [5]:
names = ['timestamp','date','query','handle','message']
df = pd.read_csv('../sentiment140.csv',encoding='Latin1',names=names)
df['sentiment'] = df.index
#Instead of positive being 4 make positive 1
df.loc[df['sentiment'] == 4,'sentiment']=1


In [6]:
df.head()

,timestamp,date,query,handle,message,sentiment
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0
0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0
0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0
0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",0


In [26]:
#Just use 1000 rows while developing the code, full set takes too long
x_data = df['message'].tolist()
y_data = df['sentiment'].tolist()

#build the vocab
x_data = [i.split() for i in x_data]
flat_list = [item for sublist in x_data for item in sublist]
vocab = utils_proj.build_vocab(flat_list)


Vocabulary: 10,000 types


In [27]:
#Determine the length of the longest tweet
max_val=0
for i in x_data:
    if len(i) > max_val:
        max_val = len(i)
print(max_val)

64


In [28]:
#canonicalize sentences, convert to ids, pad to consistant length of 100
x_ids = []
for tweet in x_data:
    temp = utils_proj.preprocess_sentences([tweet],vocab)
    x_ids.append(
        np.pad(utils_proj.preprocess_sentences([tweet],vocab),(98-len(tweet),0),'constant').tolist()) 

In [29]:
#Split into 70% train, 15% dev, 15% test

def train_test_dev(x,y,train_pct=.8,test_pct=.1,random_state=1):
    #Split into train and test
    X_train, X_test, y_train, y_test = (
        train_test_split(x, y, test_size=round(1-train_pct,1), random_state=random_state))
    #split test into dev and test
    X_dev, X_test, y_dev, y_test = (
        train_test_split(X_test, y_test, test_size=test_pct/(1-train_pct), random_state=random_state))
    
    return np.asarray(X_train),np.asarray(X_dev),np.asarray(X_test),np.asarray(y_train),np.asarray(y_dev),np.asarray(y_test)


X_train,X_dev,X_test,y_train,y_dev,y_test = train_test_dev(x_ids,y_data,.7,.15)

In [30]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=None):
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_tweets = 0
    
    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation
        
    val_acc = []
    for i, (w, y) in enumerate(batch_iterator):
        # At first batch in epoch, get a clean intitial state.
        
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})
        feed_dict = {
            lm.input_w_: w,
            lm.target_y_: y,
            lm.initial_h_: h,
            lm.learning_rate_: learning_rate,
            lm.use_dropout_: use_dropout
        }
        ops = [loss, lm.final_h_, train_op,lm.accuracy_]        
        # session.run(...) the ops with the feed_dict constructed above.
        # Ensure "cost" becomes the value of "loss".
        # Hint: see "ops" for other variables that need updating in this loop.
        cost,h,_,batch_acc = session.run(ops,feed_dict=feed_dict)
       
        val_acc.append(batch_acc)
        total_cost += cost
        total_batches = i + 1
        total_tweets += w.shape[0]  

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_tps = total_tweets / (time.time() - start_time)
            print("[batch {:d}]: seen {:d} tweets at {:.1f} tps, cost = {:.3f}".format(
                i, total_tweets, avg_tps, avg_cost))
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches, val_acc

In [31]:
def score_dataset(lm, session,ids,labels, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils_proj.rnnlm_batch_generator(ids, 100, 100,labels)
    cost,logits = run_epoch(lm, session, bi, 
                     learning_rate=0.0, train=False, 
                     verbose=False, tick_s=3600)
    print("{:s}: avg. loss: {:.03f}  (perplexity: {:.02f})".format(name, cost, np.exp(cost)))
    return cost

In [32]:
# Training parameters
max_time = 100
batch_size = 10000
learning_rate = 0.01
num_epochs = 4

# Model parameters
model_params = dict(V=vocab.size, 
                    H=200, 
                    softmax_ns=200,
                    num_layers=2,
                    num_classes=1)

TF_SAVEDIR = "/tmp/w266_final_project"
checkpoint_filename = os.path.join(TF_SAVEDIR, "lstm")
trained_filename = os.path.join(TF_SAVEDIR, "lstm_trained")

In [ ]:
print('hi')

In [ ]:
# Will print status every this many seconds
reload(lstm)
reload(utils_proj)
print_interval = 5

lm = lstm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)
    
    
    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils_proj.rnnlm_batch_generator(X_train, batch_size, max_time,y_train)

        cost,val_acc = run_epoch(lm, session, bi,True,
                  True,1.0,learning_rate)
        
        #### END(YOUR CODE) ####
        print("[epoch {:d}] Completed in {:s}".format(epoch, utils_proj.pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpointls 
        saver.save(session, checkpoint_filename, global_step=epoch)
    

    #print("[epoch {:d}]".format(epoch), end=" ")
    #score_dataset(lm, session,X_train, y_train, name="Train set")
  #  print("[epoch {:d}]".format(epoch), end=" ")
  #  score_dataset(lm, session,X_dev, y_dev, name="Dev set")
  #  print("")
    # Save final model
    saver.save(session, trained_filename)

[batch 0]: seen 10000 tweets at 172.7 tps, cost = 0.250
[batch 1]: seen 20000 tweets at 193.9 tps, cost = 0.256
[batch 2]: seen 30000 tweets at 202.3 tps, cost = 0.282
[batch 3]: seen 40000 tweets at 205.8 tps, cost = 0.275
[batch 4]: seen 50000 tweets at 208.9 tps, cost = 0.271
[batch 5]: seen 60000 tweets at 211.1 tps, cost = 0.268
[batch 6]: seen 70000 tweets at 212.5 tps, cost = 0.265
[batch 7]: seen 80000 tweets at 213.8 tps, cost = 0.263
[batch 8]: seen 90000 tweets at 214.6 tps, cost = 0.260
[batch 9]: seen 100000 tweets at 215.2 tps, cost = 0.258
[batch 10]: seen 110000 tweets at 214.1 tps, cost = 0.257
[batch 11]: seen 120000 tweets at 214.7 tps, cost = 0.256
[batch 12]: seen 130000 tweets at 215.2 tps, cost = 0.255
[batch 13]: seen 140000 tweets at 215.7 tps, cost = 0.254
[batch 14]: seen 150000 tweets at 216.2 tps, cost = 0.252
[batch 15]: seen 160000 tweets at 216.6 tps, cost = 0.250
[batch 16]: seen 170000 tweets at 216.8 tps, cost = 0.248
[batch 17]: seen 180000 tweets at

[batch 29]: seen 300000 tweets at 221.3 tps, cost = 0.141
[batch 30]: seen 310000 tweets at 221.3 tps, cost = 0.141
[batch 31]: seen 320000 tweets at 221.3 tps, cost = 0.141
[batch 32]: seen 330000 tweets at 221.3 tps, cost = 0.141
[batch 33]: seen 340000 tweets at 221.3 tps, cost = 0.141
[batch 34]: seen 350000 tweets at 221.3 tps, cost = 0.140
[batch 35]: seen 360000 tweets at 221.4 tps, cost = 0.140
[batch 36]: seen 370000 tweets at 221.4 tps, cost = 0.140
[batch 37]: seen 380000 tweets at 221.3 tps, cost = 0.140
[batch 38]: seen 390000 tweets at 221.3 tps, cost = 0.140
[batch 39]: seen 400000 tweets at 221.3 tps, cost = 0.140
[batch 40]: seen 410000 tweets at 221.3 tps, cost = 0.140
[batch 41]: seen 420000 tweets at 221.4 tps, cost = 0.140
[batch 42]: seen 430000 tweets at 221.4 tps, cost = 0.140
[batch 43]: seen 440000 tweets at 221.3 tps, cost = 0.140
[batch 44]: seen 450000 tweets at 221.3 tps, cost = 0.140
[batch 45]: seen 460000 tweets at 221.3 tps, cost = 0.140
[batch 46]: se

[batch 58]: seen 590000 tweets at 220.6 tps, cost = 0.132
[batch 59]: seen 600000 tweets at 220.6 tps, cost = 0.132
[batch 60]: seen 610000 tweets at 220.6 tps, cost = 0.132
[batch 61]: seen 620000 tweets at 220.6 tps, cost = 0.132
[batch 62]: seen 630000 tweets at 220.6 tps, cost = 0.132
[batch 63]: seen 640000 tweets at 220.6 tps, cost = 0.132
[batch 64]: seen 650000 tweets at 220.6 tps, cost = 0.132
[batch 65]: seen 660000 tweets at 220.6 tps, cost = 0.132
[batch 66]: seen 670000 tweets at 220.5 tps, cost = 0.132
[batch 67]: seen 680000 tweets at 220.6 tps, cost = 0.132
[batch 68]: seen 690000 tweets at 220.5 tps, cost = 0.132
[batch 69]: seen 700000 tweets at 220.5 tps, cost = 0.132
[batch 70]: seen 710000 tweets at 220.5 tps, cost = 0.132
[batch 71]: seen 720000 tweets at 220.5 tps, cost = 0.132
[batch 72]: seen 730000 tweets at 220.5 tps, cost = 0.132
[batch 73]: seen 740000 tweets at 220.5 tps, cost = 0.132
[batch 74]: seen 750000 tweets at 220.6 tps, cost = 0.132
[batch 75]: se

In [ ]:
# Testingg parameters
max_time = 100
batch_size = 100
learning_rate = 0.01
num_epochs = 1

# Will print status every this many seconds
reload(lstm)
reload(utils_proj)
print_interval = 5

lm = lstm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)
    
    
    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils_proj.rnnlm_batch_generator(X_test, batch_size, max_time,y_test)
        print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))
        #### YOUR CODE HERE ####
        # Run a training epoch.
        cost,val_acc = run_epoch(lm, session, bi,False,
                  True,1.0,learning_rate)
        
        #### END(YOUR CODE) ####
        print("[epoch {:d}] Completed in {:s}".format(epoch, utils_proj.pretty_timedelta(since=t0_epoch)))
    
  

    #print("[epoch {:d}]".format(epoch), end=" ")
    #score_dataset(lm, session,X_train, y_train, name="Train set")
  #  print("[epoch {:d}]".format(epoch), end=" ")
  #  score_dataset(lm, session,X_dev, y_dev, name="Dev set")
  #  print("")
    # Save final model


In [ ]:
np.mean(val_acc)